# Stress molecules

### Import path

In [ ]:
win_path = str(input('Type in the path of folder "fpilarova_masters-main" (ex. C:\\\\Users\\\\frant\\\\Desktop\\\\fpilarova_masters-main):\n'))

main_path = win_path.replace('\\\\', '\\')

### Import modules

In [ ]:
import httplib2 as http
import json
import os
import csv
import re
import math
import pandas as pd
import fastaparser
from collections import Counter
from urllib.request import urlopen
import requests
from itertools import islice



try:
    from urlparse import urlparse
except ImportError:
    from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)

## 1 Get a list of stress molecule symbols
**Description of the script:**
1. List of gene symbols is downloaded; for every pathway requested (input: Reactome ID)
2. The symbols are checked to be HGNC-approved
    - if not -> printed (EDIT - ask HGNC for HGNC-approved name of that gene)
3. Checked symbols are added to the final list (stress_molecules_list = containing stress molecules of all requested Reactome IDs)
5. Table gene_in_multiple_paths_check is created; with information about in which Reactome pathways the genes are involved

### Variables

In [ ]:
stress_molecules_list = []
#stress_molecules_list = ['AAAS', 'HSF1'] #TEST
# final list of all stress molecules; to be saved as stress_molecules.csv
      
reactome_ids_symbols_dictionary = {}
# dictionary of Reactome ids and their HGNC-approved gene symbols
# {Reactome_ID_1: [gene_1, gene_2], ...}

gene_in_multiple_paths_check = {}
# dictionary of genes and Reactome IDs in which the genes are included
# {gene_1: [Reactome_ID_1, Reactome_ID_2], ...} 

HGNC_not = []
# list of gene symbols that are not HGNC-approved

### Script: Ask user for Reactome path IDs
**Input:**
- reactome_ids = list of Reactome IDs
- reactome_names (optional)


**Testing input:**

- R-HSA-3371556 R-HSA-9612973 R-HSA-9711097 R-HSA-381119
- hsr autophagy starvation upr




In [ ]:
# ask for input

reactome_ids_input = str(input(' Enter Reactome path IDs (with spaces as separators):')) # ask for IDs
reactome_ids = reactome_ids_input.split(' ')
pathway_names_input = str(input('\n Do you want to name them? \n Yes - enter the names (with spaces as separators) \n No - press enter \n')) #ask for names if needed
pathway_names = pathway_names_input.split(' ')

while (len(reactome_ids) != len(pathway_names)) and (pathway_names[0] != ''): # check if there is the same number of names and ids
    print(' Error: there is different number of path IDs and path names.')
    reactome_ids_input = str(input(' Enter Reactome path IDs (with spaces as separators): '))
    reactome_ids = reactome_ids_input.split(' ')
    pathway_names_input = str(input('Do you want to name them? \n Yes - enter the names (with spaces as separators) \n No - press enter \n'))
    pathway_names = pathway_names_input.split(' ')


# inform about requested IDs

print('\n List of Reactome IDs saved to be examined later:') 
for i in range(len(reactome_ids)):                                           
    if pathway_names[0] != '':
        print(reactome_ids[i], ' (', pathway_names[i], ')', sep='')
    else:
        print(reactome_ids[i])

print('\n')

### Script: Download the symbols from Reactome and check if they are HGNC-approved
**Description of the script:**
1. Gene symbols from Reactome are downloaded (using Reactome IDs requested earlier)
2. The symbols are checked to be HGNC-approved
3. List stress_molecules_list is created; containing HGNC-approved symbols, each only once
4. Dictionary 'gene_in_multiple_paths_check' is generated to identify genes that are involved in multiple Reactome pathways.

Results:
1. stress_molecules_list
    - stress_molecules.csv
2. gene_in_multiple_paths_check 
    - gene_in_multiple_paths_check.csv

In [ ]:
for k in range(len(reactome_ids)): # for every Reactome ID requested:
    
    if pathway_names[0] != '':    
        print('Gene symbols of pathway: {0} (ID: {1}) are being downloaded and checked...'.format(pathway_names[k], reactome_ids[k]))
    else: 
        print('Gene symbols of pathway with ID: {} are being downloaded and checked...'.format(reactome_ids[k]))
    
    
    reactome_id = reactome_ids[k]
    
    working_symbols_list = []        # list of gene symbols of current Reactome ID to be HGNC-checked
    
    HGNC_approved_symbols_list = []
    
    
    
    # download a list of molecules using Reactome API 
    
    headers = {'Accept': 'application/json'}

    uri = 'https://reactome.org/ContentService/data/participants'
    path = '/{}'.format(reactome_id)

    target = urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
        target.geturl(),
        method,
        body,
        headers)


    if response['status'] == '200':    # assume that content is a json reply
        data = json.loads(content)     # parse content with the json module 


        for i in range(len(data)):
            for j in range(len(data[i]['refEntities'])):
                if 'UniProt' in data[i]['refEntities'][j]['displayName']:         # choose only UniProt outcomes; i.e. gene symbols
                    symbol = data[i]['refEntities'][j]['displayName'].split()[1]
                    if symbol not in working_symbols_list:                        # check that every symbol is only once
                        working_symbols_list.append(symbol)

    else:
        print( 'Error detected: ' + response['status'])
           
    
    
    # with using HGNC API find out if symbols are HGNC-approved

    if len(working_symbols_list) != 0:                 # if list is not empty
        for symbol in working_symbols_list:            # check every symbol in list
            
            # set the url
            headers = {'Accept': 'application/json'}

            uri = 'https://rest.genenames.org'
            path = '/search/symbol/{}'.format(symbol)

            target = urlparse(uri+path)
            method = 'GET'
            body = ''

            h = http.Http()

            response, content = h.request(
                target.geturl(),
                method,
                body,
                headers)

            if response['status'] == '200':            # assume that content is a json reply
                data = json.loads(content)             # parse content with the json module 
                check = data['response']['numFound']   # check can be 0 (not HGNC-approved) or 1 (approved)

                if (check == 0) and (symbol not in HGNC_not):
                    HGNC_not.append(symbol)
                elif (check == 1) and (symbol not in HGNC_approved_symbols_list):
                    HGNC_approved_symbols_list.append(symbol)

            else:
                print( 'Error detected: ' + response['status'])
        
        reactome_ids_symbols_dictionary[reactome_id] = HGNC_approved_symbols_list      # 'reactome_id_1: [gene_1, gene2, ...]' is added to dictionary
        

    else: 
        print('Error: there is no data for {}.'.format(reactome_id))
    
    for gene in HGNC_approved_symbols_list:
        if gene not in stress_molecules_list: 
            stress_molecules_list.append(gene)  # HGNC-approved symbols added to list of stress molecules (--> stress_molecules.csv)
        
        if gene not in gene_in_multiple_paths_check.keys():
            gene_in_multiple_paths_check.update({gene: [reactome_id]})
        else:
            gene_in_multiple_paths_check[gene].append(reactome_id)
        


# print symbols that are not HGNC-approved

if len(HGNC_not) != 0:    
    print('\n Symbols not HGNC-approved:', sep = ' ')
    for gene in HGNC_not:
        print(gene, sep = ' ')

        

df_stress_molecules = pd.DataFrame(stress_molecules_list, columns = ['symbol'])
    
multiple_paths_df = pd.DataFrame.from_dict(gene_in_multiple_paths_check, orient = 'index')


print('\n Done.')

### Results: List of stress molecules

In [ ]:
df_stress_molecules

### Results: Multiple paths check
To identify genes that are involved in multiple Reactome pathways.

In [ ]:
multiple_paths_df

## 2 Get orthologs for every "stress molecule"; from NCBI
**Description of the script:** 
1. Download and save FASTA files using NCBI Datasets
    - FASTA file (orthologs) for every stress molecule using NCBI Datasets (datasets.exe) is downloaded and saved
2. Parse FASTA files
    - FASTA files are parsed and saved to: 
        - orthologs_list with all information from FASTA file saved
        - orthologs_count for analysis of count of sequences per gene id
        - genes_organisms_dictionary for genes-organisms check
        - all_orthologs_refseqs_sorted for selection of one sequence per gene id
3. Genes-organisms check



### Download and save FASTA files using NCBI Datasets

#### Script

In [ ]:
print('FASTA files for every stress molecule containing data with orthologous sequences are being downloaded and saved... \n') 

for stress_molecule in stress_molecules_list:
    os.system('datasets download ortholog symbol {1} --taxon-filter mammals --exclude-rna --exclude-gene --filename {0}\\NCBI_Datasets_download\\{1}.zip'.format(win_path, stress_molecule))
    os.system('cd {0}\\NCBI_Datasets_download\\ & tar -xf {0}\\NCBI_Datasets_download\\{1}.zip'.format(win_path, stress_molecule))
    os.system('move {0}\\NCBI_Datasets_download\\ncbi_dataset\\data\\protein.faa {0}\\NCBI_Datasets_download'.format(win_path))
    os.system('ren {0}\\NCBI_Datasets_download\\protein.faa {1}.faa'.format(stress_molecule))
    os.system('del {0}\\NCBI_Datasets_download\\{1}.zip'.format(win_path, stress_molecule))
os.system('del {0}\\NCBI_Datasets_download\\README.md'.format(win_path))
os.system('rmdir /s /q {0}\\NCBI_Datasets_download\\ncbi_dataset'.format(win_path))


files = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
files.remove('.ipynb_checkpoints')


for stress_molecule in stress_molecules_list:
    if str(stress_molecule) + '.faa' not in files:
        print('Error: a file for', stress_molecule, 'was not downloaded. \n')
        stress_molecules_list.remove(stress_molecule)

print('Done.')


### Parse FASTA files

#### Variables and functions

In [ ]:
# GENERAL VARIABLES
gene_ids = [] 
# gene ids of all orthologs (of all stress molecules together)

orthologs_list = [] 
# list of all orthologous sequences and information about them
# [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession_number (that is refseq number): X, protein sequence: X}, {...}]
# -> orthologs.csv


# TO SEE HOW MANY REFERENCE SEQUENCES THERE ARE FOR ONE GENE ID
orthologs_count_list = []
# list of genes and count of its sequences (refseq) downloaded from NCBI 
# [{human_gene_name: x, gene_id: x, seq_count = x, most_repres_seq_count = x, unique_seq_count = x}, {...}, ...]

already_been_there = []
# working list


# FOR GENES-ORGANISMS CHECK
genes_organisms_dictionary = {}



# FOR SELECTION OF ONE SEQUENCE PER GENE ID
all_orthologs_refseqs_sorted = {}
# {gene_id1: 
#     {RefSeq1: 
#          {protein_sequence: X, human_gene_name: X, gene_name: X, organism: X}, 
#      RefSeq2: {...}
#     }, 
#  gene_id2: 
#      {RefSeq3: {}
#     }, 
#  ...}

In [ ]:
def most_frequent_count(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][1]

def most_frequent_name(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

#### Script: Parse FASTA files and save the information into lists and dictionaries

In [ ]:
# parse all FASTA files downloaded 
# every file is named as the gene which orthologs seqs it contains of

files = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
files.remove('.ipynb_checkpoints')

for file in files:
    
    gene_name = file.replace('.faa', '')

    orthologs_count_dictionary = {}   # working dictionary; will be used for "seqs count check" 
                                      # {gene_id1: [seq1, seq2, seq3], gene_id2: [seq1, seq2], ...}
    
    organism_gene_id_dict = {}        # working dictionary; will be used for genes-organisms check
        
    with open(r'{0}\NCBI_Datasets_download\{1}'.format(main_path, file)) as fasta_file:
        parser = fastaparser.Reader(fasta_file)
        
        for seq in parser: # for every record (ortholog) of FASTA file        
            
            # split the FASTA record 
            working_list = seq.description.split(" [")     
            gene_id = (re.findall('\\d+', working_list[2])[0])
            gene_ids.append(gene_id)


            
            ### for orthologs.csv ###
            # add seq (ortholog) atributes to orthologs_list
            
            ortholog_dictionary = {}   # {human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X}
            
            ortholog_dictionary['human_gene_name'] = gene_name                              # human gene name added
            ortholog_dictionary['gene_name'] = working_list[0]                              # gene name added
            organism_name = working_list[1][9:].replace("]", "")
            organism_name = organism_name.replace(" ", "_")
            ortholog_dictionary['organism'] = organism_name                                 # organism name added
            ortholog_dictionary['gene_id'] = gene_id                                        # gene ID added
            ortholog_dictionary['accession_number'] = seq.id                                # accession number added 
            ortholog_dictionary['protein_sequence'] = seq.sequence_as_string()              # protein sequence with its accession number added

            orthologs_list.append(ortholog_dictionary)
            # orthologs_list = [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X}, {...}]
            
            

            ### for orthologs_count.csv and final_orthologs_seqs.csv ###     
            # add sequence to orthologs_count_dictionary {gene_id1: [seq1, seq2, seq3], gene_id2: [seq1, seq2], ...}
            # add RefSeq number and sequence to all_orthologs_refseqs_sorted dictionary {gene_id1: {RefSeq1: seq1, RefSeq2: seq2, ...}, gene_id2: {}, ...}
            
            if gene_id not in already_been_there: 
                first_sequence = seq.sequence_as_string()
                orthologs_count_dictionary[gene_id] = [first_sequence]
                all_orthologs_refseqs_sorted[gene_id] = {}
                all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': working_list[1][9:].replace("]", "")}})
                
            else:
                another_sequence = seq.sequence_as_string()
                orthologs_count_dictionary[gene_id].append(another_sequence)
                all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': working_list[1][9:].replace("]", "")}})

            already_been_there.append(gene_id)
            
            
            
            ### for genes_organisms_check.csv ###
            organism_gene_id_dict[organism_name] = gene_id      

            
        genes_organisms_dictionary.update({gene_name : organism_gene_id_dict})
        # {gene_name1: {organism1: gene_id1, organism2: gene_id2, ...}, gene_name2: {}, ...}
            
            
    
    ### for orthologs_count.csv ###     
    # create dictionary with gene ids and their protein sequence counts (and the most represented sequence)
    
    for key in orthologs_count_dictionary.keys():
        ortholog = {}
        ortholog['human_gene_name'] = file.strip('.faa')
        ortholog['gene_id'] = str(key)
        ortholog['seq_count'] = len(orthologs_count_dictionary[key])
        ortholog['most_repres_seq_count'] = most_frequent_count(orthologs_count_dictionary[key])
        
        counter_keys = Counter(orthologs_count_dictionary[key])
        ortholog['unique_seq_count'] = len(counter_keys.keys())
        
        orthologs_count_list.append(ortholog)
        # orthologs_count_list = [{human_gene_name: 'xxx.faa', gene_id: x, seq_count = x, most_repres_seq_count = x, unique_seq_count = x}, {...}, ...]


df_orthologs_list = pd.DataFrame.from_dict(orthologs_list)
df_ortholog_count_list = pd.DataFrame.from_dict(orthologs_count_list)



#### Results: List of all orthologs downloaded 

In [ ]:
df_orthologs_list

#### Results: Count of reference sequences of every orthologous gene


In [ ]:
df_ortholog_count_list

### Genes-organisms check


#### Script

In [ ]:
# create DataFrame from genes_organisms_dictionary

df_genes_organisms = pd.DataFrame(list(genes_organisms_dictionary.values()), index=list(genes_organisms_dictionary.keys()))

# variables

genes_count = len(genes_organisms_dictionary.keys())
organisms_count = 0
count_organisms_seq_ratio = {}
count_genes_seq_ratio = {}

# for which organisms there is enough data? 
# i.e. ratio of nonexisting vs existing ortholog seqs for every organism
for column in df_genes_organisms:
    organisms_count += 1
    count_organisms_seq_ratio.update({column: df_genes_organisms[column].isnull().sum()/genes_count})

# for which genes there is enough data? 
# i.e. ratio of nonexisting vs existing seqs for every gene
for key in genes_organisms_dictionary.keys():
    nan_count = df_genes_organisms.loc[[key]].isna().sum().sum()
    count_genes_seq_ratio.update({key: nan_count/organisms_count})



# pop an organism or a gene from the DataFrame if there is not enough data (seqs) for them 

organisms_not_enough = []
genes_not_enough = []
gene_ids_to_pop = [] # list of gene ids to pop from all_orthologs_refseq_sorted (aka list for future analysis)

for key in count_organisms_seq_ratio.keys():
    if count_organisms_seq_ratio[key] > 0.2:
        organisms_not_enough.append(key)
        

# Remove the organism with too litle data from the analysis
for organism in organisms_not_enough:
    gene_ids_to_pop += df_genes_organisms[organism].to_list()
    df_genes_organisms.pop(organism)
    
    

for key in count_genes_seq_ratio.keys():
    if count_genes_seq_ratio[key] > 0.2:
        genes_not_enough.append(key)
        
        
# Remove the gene with too litle data from the analysis
for gene in genes_not_enough:
    gene_ids_to_pop += df_genes_organisms.loc[gene].tolist()
    df_genes_organisms.drop(gene, inplace=True)



# print genes and organisms which enough data do not exist for

if len(genes_not_enough) != 0:
    print('There is not enough data for genes:')
    for gene in genes_not_enough:
        print(gene)

if len(organisms_not_enough) != 0:
    print('There is not enough data for organisms:')
    for organism in organisms_not_enough:
        print(organism)
        
print('\n They are not included in future analysis.')



# remove organisms and genes with not enough data from all_orthologs_refseqs_sorted
gene_ids_to_pop = [b for b in gene_ids_to_pop if isinstance(b, str)] # remove all nan


for geneid in gene_ids_to_pop:
    del all_orthologs_refseqs_sorted[geneid]



#### Results: Final table of genes, organism and corresponding gene ids

In [ ]:
df_genes_organisms

## 3 Animal Traits & AnAge databases; calculating longevity quotient

**Description of script:** Edits data from Animal Traits dataset and saves name of an organism and its body mass to table 'df_animaltraits'. Merges AnAge data and Animal Traits data (table 'df_anage_animaltraits'); and calculate longevity quotient. 

#### Script: Load Animal Traits dataset

In [ ]:
df_animaltraits = pd.read_csv(r'{0}\AnimalTraits_dataset\AnimalTraits.csv'.format(main_path), sep=';')

animaltraits_bodymass = {}

for index, rows in df_animaltraits.iterrows():
        name = str(rows.species)
        if 'e_' in str(rows.body_mass): 
            main_num = str(rows.body_mass).split('e_0')
            body_mass = float(main_num[0]) / (10 ** float(main_num[1])) * 1000 
            df_animaltraits.at[index,'body_mass'] = body_mass
            
        else: 
            body_mass = round(float(rows.body_mass) * 1000, 2)
            df_animaltraits.at[index,'body_mass'] = body_mass
        animaltraits_bodymass.update({name: body_mass})
        
df_animaltraits = df_animaltraits.iloc[:, [5, 12]]

#### Results: List of organisms and their body mass (in g) from Animal Traits dataset

In [ ]:
df_animaltraits

### Script: Fill in missing body masses in AnAge with data from Animal Traits

In [ ]:
def Sort(sub_li):
    return(sorted(sub_li, key = lambda x: x[1]))   

In [ ]:
df_anage = pd.read_csv(r'{0}\AnAge_dataset\anage_data.csv'.format(main_path), sep=';')
organisms_downloaded = df_genes_organisms.columns.values.tolist()
  
for index, rows in df_anage.iterrows():
        name = str(rows.Genus) + '_' + str(rows.Species)
        if math.isnan(rows.Body_mass):
            if name in animaltraits_bodymass.keys():
                df_anage.at[index,'Body_mass'] = animaltraits_bodymass[name]
                
df_anage_animaltraits = df_anage.iloc[:, [6,7,20,28]]

### Results: AnAge and Animal Traits merged; table containing max. longevity and body mass

In [ ]:
df_anage_animaltraits

### Script: Calculate longevity quotient and choose low and top decile (aka short and long-living species)

In [ ]:
organisms_in_organisms_lq_list =[]
organisms_lq_list_all = []
organisms_lq_list_final = []

# Go through the AnAge + Animal Traits table and calculate longevity quotient for each organism

for index, rows in df_anage_animaltraits.iterrows():
        name = str(rows.Genus) + '_' + str(rows.Species)
        if not math.isnan(rows.Body_mass) and not math.isnan(rows.Maximum_longevity):
           
            # maximum longevity
            ml = float(rows.Maximum_longevity)
            
            # body mass
            bm = float(rows.Body_mass)

            # longevity quotient
            lq = round(ml/(6.32*(bm**0.139)), 2)
            
            
            
            # All organisms from AnAge and Animal Traits datasets that data on body mass and lifespan exist
            
            organisms_lq_list_all.append({'organism':name, 'maximum_lifespan':ml, 'body_mass':bm, 'longevity_quotient':lq})
            
            
            
            # Overlap between list of organisms, that we have data for, and list of organisms with known bodymasses and lifespans
            
            if name in organisms_downloaded: 
                organisms_lq_list_final.append({'organism':name, 'maximum_lifespan':ml, 'body_mass':bm, 'longevity_quotient':lq})
        
        
# FINAL table of all organisms that data for orthologs and longevity quotient are available  
            
df_organisms_lq = pd.DataFrame.from_dict(organisms_lq_list_final)



# Calculate low and top decile of longevity quotients and save them as short and long-living organisms

df_organisms_lq['decile_rank'] = pd.qcut(df_organisms_lq['longevity_quotient'], 10, labels = False)

df_short_living = df_organisms_lq.loc[df_organisms_lq['decile_rank'].isin([0])]
short_living_list = df_short_living['organism'].to_list()

df_long_living = df_organisms_lq.loc[df_organisms_lq['decile_rank'].isin([9])]
long_living_list = df_long_living['organism'].to_list()


### Results: List of organisms sorted by decile rank
Low decile - short-living organisms

Top decile - long-living organisms

In [ ]:
df_organisms_lq.sort_values('decile_rank')

### Results: List of short-living organisms

In [ ]:
df_short_living

### Results: List of long-living organisms

In [ ]:
df_long_living

### Script: Create dictionaries to choose one seq per geneid for selected long and short-living organisms

In [ ]:
# for long-living organisms, all orthologous sequences sorted by geneid and refseq
long_orthologs_refseqs_sorted = {} 

geneids_long = []

for organism in long_living_list: 
    geneids_long += df_genes_organisms[organism].to_list()

geneids_long = [b for b in geneids_long if isinstance(b, str)] # remove all nan

for geneid in geneids_long:
     long_orthologs_refseqs_sorted.update({geneid:all_orthologs_refseqs_sorted[geneid]})
    

# for short-living organisms, all orthologous sequences sorted by geneid and refseq
short_orthologs_refseqs_sorted = {}

geneids_short = []

for organism in short_living_list:
    geneids_short += df_genes_organisms[organism].to_list()

geneids_short = [b for b in geneids_short if isinstance(b, str)] # remove all nan

for geneid in geneids_short:
    short_orthologs_refseqs_sorted.update({geneid:all_orthologs_refseqs_sorted[geneid]})

## 4 SHORT-living; Select reference sequences 
**Description of script:** It chooses which protein sequence will be used as a reference sequence.

Sequence is chosen as a reference if there are keywords 'RefSeq Select' in GenPept file of the protein - script asks NCBI API for GenPept file and check it. Else the longest sequence from all protein sequences of that gene is chosen.

seq_selection: 
  1. 'only_one_seq'; there was only one reference sequence in NCBI dtb
  2. 'RefSeq_Select'; RefSeq Select in keywords of GenPept file
  3. 'longest_seq'; the longest protein sequence


### Variables and functions

In [ ]:
short_selected_orthologs_list = [] 
# = list made of the 'orthologs_list' but containing only one seq per gene id
# + information if it was chosen with RefSeq Select keyword or it is the longest sequence
# [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X, seq_selection: X}, {...}]


# testing dictionary 
# short_orthologs_refseqs_sorted = {'gene_id1': {'NP_001104745': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}, 'XP_023505037.1': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}, 'gene_id2': {'XP_023505041.1' :{'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}}


In [ ]:
def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

### Script: Parse dictionary short_orthologs_refseqs_sorted

In [ ]:
#all refseqs from dictionary short_orthologs_refseqs_sorted

all_refseqs_list = [] 
gene_ids_list = []

for key in list(short_orthologs_refseqs_sorted.keys()):
    gene_ids_list.append(key)
    for jey in list(short_orthologs_refseqs_sorted[key].keys()): 
        all_refseqs_list.append(jey)
        
print('Count of all gene ids:', len(gene_ids_list))
print('Count of all refseqs:', len(all_refseqs_list), '\n')



# split the dictionary by 50 
splitted_short_orthologs_refseqs_sorted = []


for item in chunks({i:short_orthologs_refseqs_sorted[i] for i in short_orthologs_refseqs_sorted}, 10):
    splitted_short_orthologs_refseqs_sorted.append(item)
    
    
count_list = []
count = 0
splitted_refseqs_list = []
for item in splitted_short_orthologs_refseqs_sorted:
    for key in list(item.keys()):
        
        for jey in list(item[key].keys()):
            count+=1
            splitted_refseqs_list.append(jey)
    count_list.append(count)
    count = 0

print('Count in splitted:', len(splitted_refseqs_list))


print('Number of sub-parts of short_orthologs_refseqs_sorted to be examined:', len(splitted_short_orthologs_refseqs_sorted))
# print('Counts of refseqs in subparts:', count_list)



### Script: Selection - one protein seq per gene id

In [ ]:
for chunk in splitted_short_orthologs_refseqs_sorted:
    
    
    refseqs_string = '' 
    refseqs_list = []
    
    
    for key in list(chunk.keys()):
        for jey in list(chunk[key].keys()): 
            refseqs_string = refseqs_string + '&id=' + jey
            refseqs_list.append(jey)
                    
    
    urls = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein{}&api_key=aabfba1a0deb347c0ffdd37c6e46908c6a09&rettype=gp".format(refseqs_string), auth=('user', 'pass'), timeout=(3.05, 27))
    genpept_files = urls.text.split('//')
    if '\n\n' in genpept_files: genpept_files.remove('\n\n')

    # print('Number of downloaded files is the same as number of RefSeq numbers:', len(genpept_files) == len(refseqs_list))
    # print(len(genpept_files), len(refseqs_list))
    
    count = -1
    for geneid, refseqs_of_geneid in chunk.items(): 
        refseq_select = None
        the_longest_seq = None
        seq_len = 0

        refseqs = list(refseqs_of_geneid.keys())
      
        # if there is more than one RefSeq sequence for gene id
        if len(refseqs) > 1 and count < len(genpept_files):
            for refseq in refseqs:
                count += 1
                genpept_file = genpept_files[count]
                if refseq not in genpept_file: print('Error: File does not correspond to RefSeq number.')

                # if the reference sequence is RefSeq Select, save it
                if 'RefSeq Select' in genpept_file:    
                    refseq_select = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'RefSeq_Select'}


                # save the longest reference sequence 
                if len(refseqs_of_geneid[refseq]['protein_sequence']) > seq_len: 
                    seq_len = len(refseqs_of_geneid[refseq]['protein_sequence'])
                    the_longest_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'longest_seq'}


            if refseq_select: 
                short_selected_orthologs_list.append(refseq_select)

            elif the_longest_seq: 
                short_selected_orthologs_list.append(the_longest_seq)  
            

        else: # there is only one RefSeq
            count += 1
            refseq = list(refseqs_of_geneid.keys())[0]
            only_one_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'only_one_seq'}
            short_selected_orthologs_list.append(only_one_seq) 



df_short_selected_orthologs_list = pd.DataFrame.from_dict(short_selected_orthologs_list)

### Results: Short-living organisms; table of selected reference sequences; one per gene id

In [ ]:
df_short_selected_orthologs_list

## 4 LONG-living; Select reference sequences 
**Description of script:** It chooses which protein sequence will be used as a reference sequence.

Sequence is chosen as a reference if there are keywords 'RefSeq Select' in GenPept file of the protein - script asks NCBI API for GenPept file and check it. Else the longest sequence from all protein sequences of that gene is chosen.

seq_selection: 
  1. 'only_one_seq'; there was only one reference sequence in NCBI dtb
  2. 'RefSeq_Select'; RefSeq Select in keywords of GenPept file
  3. 'longest_seq'; the longest protein sequence


### Variables and functions

In [ ]:
long_selected_orthologs_list = [] 
# = list made of the 'orthologs_list' but containing only one seq per gene id
# + information if it was chosen with RefSeq Select keyword or it is the longest sequence
# [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X, seq_selection: X}, {...}]


# testing dictionary 
# all_orthologs_refseqs_sorted = {'gene_id1': {'NP_001104745': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}, 'XP_023505037.1': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}, 'gene_id2': {'XP_023505041.1' :{'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}}


In [ ]:
def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

### Script: Parse dictionary long_orthologs_refseqs_sorted

In [ ]:
#all refseqs from dictionary long_orthologs_refseqs_sorted

all_refseqs_list = [] 
gene_ids_list = []

for key in list(long_orthologs_refseqs_sorted.keys()):
    gene_ids_list.append(key)
    for jey in list(long_orthologs_refseqs_sorted[key].keys()): 
        all_refseqs_list.append(jey)
        
print('Count of all gene ids:', len(gene_ids_list))
print('Count of all refseqs:', len(all_refseqs_list), '\n')



# split the dictionary by 50 
splitted_long_orthologs_refseqs_sorted = []


for item in chunks({i:long_orthologs_refseqs_sorted[i] for i in long_orthologs_refseqs_sorted}, 10):
    splitted_long_orthologs_refseqs_sorted.append(item)
    
    
count_list = []
count = 0
splitted_refseqs_list = []
for item in splitted_long_orthologs_refseqs_sorted:
    for key in list(item.keys()):
        
        for jey in list(item[key].keys()):
            count+=1
            splitted_refseqs_list.append(jey)
    count_list.append(count)
    count = 0

print('Count in splitted:', len(splitted_refseqs_list))


print('Number of sub-parts of long_orthologs_refseqs_sorted to be examined:', len(splitted_long_orthologs_refseqs_sorted))
# print('Counts of refseqs in subparts:', count_list)



### Script: Selection - one protein seq per gene id

In [ ]:
for chunk in splitted_long_orthologs_refseqs_sorted:
    
    refseqs_string = '' 
    refseqs_list = []
    
    
    for key in list(chunk.keys()):
        for jey in list(chunk[key].keys()): 
            refseqs_string = refseqs_string + '&id=' + jey
            refseqs_list.append(jey)
                    
    
    urls = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein{}&api_key=aabfba1a0deb347c0ffdd37c6e46908c6a09&rettype=gp".format(refseqs_string), auth=('user', 'pass'), timeout=(10, 27))
    genpept_files = urls.text.split('//')
    if '\n\n' in genpept_files: genpept_files.remove('\n\n')
    print(genpept_files)

    # print('Number of downloaded files is the same as number of RefSeq numbers:', len(genpept_files) == len(refseqs_list))
    # print(len(genpept_files), len(refseqs_list))

    
    count = -1
    for geneid, refseqs_of_geneid in chunk.items(): 
        refseq_select = None
        the_longest_seq = None
        seq_len = 0

        refseqs = list(refseqs_of_geneid.keys())
      
        # if there is more than one RefSeq sequence for gene id
        if len(refseqs) > 1 and count < len(genpept_files):
            for refseq in refseqs:
                count += 1
                print(refseq)
                print(count)
                genpept_file = genpept_files[count]
                print(genpept_file)
                if refseq not in genpept_file: print('Error: File does not correspond to RefSeq number.')

                # if the reference sequence is RefSeq Select, save it
                if 'RefSeq Select' in genpept_file:    
                    refseq_select = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'RefSeq_Select'}


                # save the longest reference sequence 
                if len(refseqs_of_geneid[refseq]['protein_sequence']) > seq_len: 
                    seq_len = len(refseqs_of_geneid[refseq]['protein_sequence'])
                    the_longest_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'longest_seq'}


            if refseq_select: 
                long_selected_orthologs_list.append(refseq_select)

            elif the_longest_seq: 
                long_selected_orthologs_list.append(the_longest_seq)  
            

        else: # there is only one RefSeq
            count += 1
            refseq = list(refseqs_of_geneid.keys())[0]
            only_one_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'only_one_seq'}
            long_selected_orthologs_list.append(only_one_seq) 



df_long_selected_orthologs_list = pd.DataFrame.from_dict(long_selected_orthologs_list)

### Results: Long-living organisms; table of selected reference sequences; one per gene id

In [ ]:
df_long_selected_orthologs_list

## 5 Divide files from NCBI Datasets into subparts (NCBI connection error)

### Script

In [ ]:
folder = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
folder.remove('.ipynb_checkpoints')
name = 1
counter = 0


os.system('mkdir {0}\\NCBI_Datasets_download\\{1}'.format(str(win_path, name)))
          
for gene in folder:
    if counter < 20:
        os.system('move {0}\\NCBI_Datasets_download\\{1} {0}\\NCBI_Datasets_download\\{2}'.format(win_path, gene, str(name)))
        counter += 1
        
    else:
        name += 1
        os.system('mkdir {0}\\NCBI_Datasets_download\\{1}'.format(str(win_path, name)))
        counter = 0
        os.system('move {0}\\NCBI_Datasets_download\\{1} {0}\\NCBI_Datasets_download\\{2}'.format(win_path, gene, str(name)))
        counter += 1

## 6 Create files for CAASTools Discovery analysis

### Script: Create config file with organisms and 0 (short-living) / 1 (long-living)

In [ ]:
config_file = open(r'{0}\CAASTools_files\config.txt'.format(main_path),"w")

for organism in short_living_list: 
    config_file.write(organism.lower() + '\t' + '0' + '\n')
for organism in long_living_list: 
    config_file.write(organism.lower() + '\t' + '1' + '\n')

config_file.close()

### Script: Create fasta file for clustal omega multiple sequence alignment

In [ ]:
all_seq = long_selected_orthologs_list + short_selected_orthologs_list
genes = df_genes_organisms.index.values.tolist()

for gene in genes:
    gene_file = open(r'{0}\CAASTools_files\fasta_files\{1}.fasta'.format(main_path, gene),"w")

    for item in all_seq:
        if gene in item.values(): 
            gene_file.write('>' + str(item['organism']).replace(' ', '_').lower() + '\n')
            gene_file.write(str(item['protein_sequence']) + '\n')

    gene_file.close

### Script: MSAs by Clustal omega
!!! It is needed to download the Clustal Omega program and save the folder "clustal-omega-1.2.2-win64" with the program "clustalo.exe" to main folder 

In [ ]:
for gene in genes:
    os.system('{0}\\clustal-omega-1.2.2-win64\\clustalo.exe -i {0}\\CAASTools_files\\fasta_files\\{1}.fasta --out {0}\\CAASTools_files\\MSA_results\\{1}.msa --outfmt clu --wrap=10000'.format(win_path, gene))

## Create csv files

### stress_molecules.csv
table of stress molecules (downloaded from Reactome - with Reactome IDs requested, and checked with HGNC)

In [ ]:
df_stress_molecules.to_csv(r'{0}\stress_molecules\stress_molecules.csv'.format(main_path), sep=',', encoding='utf-8')

### multiple_paths_check.csv
table of gene symbols and Reactome IDs; i.e. some genes may be involved in multiple paths

In [ ]:
multiple_paths_df.to_csv(r'{0}\results\gene_in_multiple_paths_check.csv'.format(main_path).format(main_path), sep=',', encoding='utf-8')

### orthologs.csv 
table of orthologs and their atributes (i.e. FASTA file parsed into table)

In [ ]:
df_orthologs_list.to_csv(r'{0}\NCBI_Datasets_download\orthologs.csv'.format(main_path), sep=',', encoding='utf-8')      

### orthologs_count.csv
table of gene ids and their protein sequence counts

In [ ]:
df_ortholog_count_list.to_csv(r'{0}\results\orthologs_count.csv'.format(main_path), sep=',', encoding='utf-8')

### genes_organisms_check.csv
table of genes-organisms check

In [ ]:
df_genes_organisms.to_csv(r'{0}\results\genes_organisms_check.csv'.format(main_path), sep=',', encoding='utf-8')

### anage_animaltraits_data_merge.csv

In [ ]:
df_anage_animaltraits.to_csv(r'{0}\results\anage_animaltraits_data_merge.csv'.format(main_path), sep=',', encoding='utf-8')

### organisms_lq.csv

In [ ]:
df_organisms_lq.to_csv(r'{0}\results\organisms_lq.csv'.format(main_path), sep=',', encoding='utf-8')

### short_living_organisms.csv

In [ ]:
df_short_living.to_csv(r'{0}\results\short_living_organisms.csv'.format(main_path), sep=',', encoding='utf-8')

### long_living_organisms.csv

In [ ]:
df_long_living.to_csv(r'{0}\results\long_living_organisms.csv'.format(main_path), sep=',', encoding='utf-8')

### long_selected_orthologs.csv
table of selected sequences one reference sequence per gene id (i.e. orthologous gene)

In [ ]:
df_long_selected_orthologs_list.to_csv(r'{0}\results\long_selected_orthologs.csv'.format(main_path), sep=',', encoding='utf-8')

### short_selected_orthologs.csv
table of selected sequences one reference sequence per gene id (i.e. orthologous gene)

In [ ]:
df_short_selected_orthologs_list.to_csv(r'{0}\results\short_selected_orthologs.csv'.format(main_path), sep=',', encoding='utf-8')